# **Analisi Margini & Ricarichi v1_0**
## **Data Analysis Notebook**
***A Jupyther Notebook by Cristiano Esposito - ©2019***<br><br>
**Dalla analisi dei prezzi di listino di un catalogo, individuare le relative marginature e determinare i ricarichi da applicare al listino di vendita, per ottenere marginature pari al 20%-30%-35%**

## **Sorgenti**
<li>summary: File processato dal notebook 01.Data Prep</li>

## **Changelog**
<li>02-09-2019: Inizio del progetto</li>
<li>03-09-2019: Inserimento origine dati</li>
<li>04-09-2019: Ottimizzazione mining dei dati</li>

## **Imports** 
Iniziamo importando le librerie python che andremo ad utilizzare nel progetto:<br>
**pandas** --> caricamento e gestione dati<br>
**numpy** --> operazioni logiche e matematiche sui dati<br>
**pathlib** --> operazioni sulla gestione dei path di file<br>
**datetime** --> libreria di operazioni sulle date

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

In [2]:
## **File Locations**

In [3]:
# Collegamento al file esterno

today = datetime.today()
in_file = Path.cwd() / "data" / "processed" / f"summary_listini.pkl"
report_dir = Path.cwd() / "reports"
report_file = report_dir / f"Ricarico_Listini_{today:%b-%d-%Y}.xlsx"

## **Elaborazione Dati**
In questa sezione elaboriamo i dati letti dal file per ottenere i KPI oggetto della nostra analisi

In [4]:
# Creiamo un dataframe attraverso il file di collegamento ottenuto dal notebook precedente

df = pd.read_pickle(in_file)

In [5]:
# Controlliamo che le colonne siano corrette

df.columns

Index(['COD.FAMIGLIA WEB', 'DES.FAMIGLIA WEB', 'LISTINO  1', 'SC.1 LIS.  1',
       'LISTINO  5', 'SC.1 LIS.  5', 'LISTINO  6', 'SC.1 LIS.  6'],
      dtype='object')

In [6]:
# Creiamo nel dataframe nuove colonne con i prezzi scontati che serviranno per il calcolo dei margini

df['LISTINO1_SCONTATO'] = df['LISTINO  1'] * (1 - df['SC.1 LIS.  1'] / 100) 
df['LISTINO5_SCONTATO'] = df['LISTINO  5'] * (1 - df['SC.1 LIS.  5'] / 100)
df['LISTINO6_SCONTATO'] = df['LISTINO  6'] * (1 - df['SC.1 LIS.  6'] / 100)

In [7]:
df

,COD.FAMIGLIA WEB,DES.FAMIGLIA WEB,LISTINO 1,SC.1 LIS. 1,LISTINO 5,SC.1 LIS. 5,LISTINO 6,SC.1 LIS. 6,LISTINO1_SCONTATO,LISTINO5_SCONTATO,LISTINO6_SCONTATO
0,A27,Sanitari,103.7561,0.0,115.5677,0.0,138.6812,0.0,103.7561,115.5677,138.6812
1,A27,Sanitari,226.6082,0.0,252.4052,0.0,302.8862,0.0,226.6082,252.4052,302.8862
2,144,Filtro e Raccordi Gas,0.3000,0.0,0.3543,0.0,0.4252,0.0,0.3000,0.3543,0.4252
3,144,Filtro e Raccordi Gas,0.5625,0.0,0.6641,0.0,0.7969,0.0,0.5625,0.6641,0.7969
4,924,Piatti Doccia,28.7500,0.0,33.2851,0.0,36.6136,0.0,28.7500,33.2851,36.6136
...,...,...,...,...,...,...,...,...,...,...,...
6083,475,Contatori e Accessori,5.2360,0.0,6.3644,0.0,7.0008,0.0,5.2360,6.3644,7.0008
6084,144,Filtro e Raccordi Gas,5.9840,0.0,7.2737,0.0,8.0011,0.0,5.9840,7.2737,8.0011
6085,235,Termocamino,12.2183,0.0,14.8514,0.0,17.8217,0.0,12.2183,14.8514,17.8217
6086,235,Termocamino,19.0118,0.0,23.1089,0.0,27.7307,0.0,19.0118,23.1089,27.7307


In [8]:
# Creiamo due tabelle pivot con la somma e la media delle colonne dei listini, raggruppando i dati in per categoria merceologica

listini_sum = pd.pivot_table(df, index=[df['COD.FAMIGLIA WEB'], df['DES.FAMIGLIA WEB']], values=['DES.FAMIGLIA WEB', 'LISTINO1_SCONTATO', 'LISTINO5_SCONTATO', 'LISTINO6_SCONTATO'], aggfunc='sum')
listini_mean = pd.pivot_table(df, index=[df['COD.FAMIGLIA WEB'], df['DES.FAMIGLIA WEB']], values=['DES.FAMIGLIA WEB', 'LISTINO1_SCONTATO', 'LISTINO5_SCONTATO', 'LISTINO6_SCONTATO'], aggfunc='mean')

In [9]:
# Andiamo a trovare i margini dei listini che sono oggetto di analisi, secondo la formula
# MARGINE_PERC = (1 - PREZZO_COSTO / PREZZO_LISTINO) * 100

listini_sum['MARGINE_LISTINO5'] = round((1 - listini_sum['LISTINO1_SCONTATO'] / listini_sum['LISTINO5_SCONTATO']) * 100, 2)
listini_sum['MARGINE_LISTINO6'] = round((1 - listini_sum['LISTINO1_SCONTATO'] / listini_sum['LISTINO6_SCONTATO']) * 100, 2)

In [10]:
# Passiamo quindi a calcolare i ricarichi per ottenere i margini desiderati, secondo la formula
# RICARICO_PERC = (PREZZO_COSTO - (1 - MARGINE_DESIDERATO / 100) * PREZZO_LISTINO) / ((1 - MARGINE_DESIDERATO / 100) * PREZZO_LISTINO) * 100

# Su listino 5
listini_sum['RIC_20_LIST_5'] = round((listini_sum['LISTINO1_SCONTATO'] - 0.80 * listini_sum['LISTINO5_SCONTATO']) / (0.80 * listini_sum['LISTINO5_SCONTATO']) * 100, 0) #margine obiettivo del 20%
listini_sum['RIC_30_LIST_5'] = round((listini_sum['LISTINO1_SCONTATO'] - 0.70 * listini_sum['LISTINO5_SCONTATO']) / (0.70 * listini_sum['LISTINO5_SCONTATO']) * 100, 0) #margine obiettivo del 30%
listini_sum['RIC_35_LIST_5'] = round((listini_sum['LISTINO1_SCONTATO'] - 0.65 * listini_sum['LISTINO5_SCONTATO']) / (0.65 * listini_sum['LISTINO5_SCONTATO']) * 100, 0) #margine obiettivo del 35%

# Su listino 6
listini_sum['RIC_20_LIST_6'] = round((listini_sum['LISTINO1_SCONTATO'] - 0.80 * listini_sum['LISTINO6_SCONTATO']) / (0.80 * listini_sum['LISTINO6_SCONTATO']) * 100, 0) #margine obiettivo del 20%
listini_sum['RIC_30_LIST_6'] = round((listini_sum['LISTINO1_SCONTATO'] - 0.70 * listini_sum['LISTINO6_SCONTATO']) / (0.70 * listini_sum['LISTINO6_SCONTATO']) * 100, 0) #margine obiettivo del 30%
listini_sum['RIC_35_LIST_6'] = round((listini_sum['LISTINO1_SCONTATO'] - 0.65 * listini_sum['LISTINO6_SCONTATO']) / (0.65 * listini_sum['LISTINO6_SCONTATO']) * 100, 0) #margine obiettivo del 35%

In [11]:
listini_sum

,,LISTINO1_SCONTATO,LISTINO5_SCONTATO,LISTINO6_SCONTATO,MARGINE_LISTINO5,MARGINE_LISTINO6,RIC_20_LIST_5,RIC_30_LIST_5,RIC_35_LIST_5,RIC_20_LIST_6,RIC_30_LIST_6,RIC_35_LIST_6
COD.FAMIGLIA WEB,DES.FAMIGLIA WEB,,,,,,,,,,,
001,Collettori per Sanitari,330.499200,385.95520,443.748200,14.37,25.52,7.0,22.0,32.0,-7.0,6.0,15.0
003,Accessori Collettori,4.035800,4.71100,5.377200,14.33,24.95,7.0,22.0,32.0,-6.0,7.0,15.0
004,Collettore Riscaldamento,549.835100,648.04370,756.363400,15.15,27.31,6.0,21.0,31.0,-9.0,4.0,12.0
005,Collettori per Contatori,378.822500,438.53450,504.314800,13.62,24.88,8.0,23.0,33.0,-6.0,7.0,16.0
006,Collettori Refrigerazione,114.611200,139.31060,167.172700,17.73,31.44,3.0,18.0,27.0,-14.0,-2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
C55,Accessori Sanitari,140.027224,163.53762,186.028188,14.38,24.73,7.0,22.0,32.0,-6.0,8.0,16.0
C56,Tappo Piletta,4.363900,5.30440,5.834800,17.73,25.21,3.0,18.0,27.0,-7.0,7.0,15.0
C88,Accessori per Lavoro,284.915200,349.49060,379.942100,18.48,25.01,2.0,16.0,25.0,-6.0,7.0,15.0


In [12]:
# Copiamo i dati così ottenuti in nuovi dataframes

margini = listini_sum[['MARGINE_LISTINO5', 'MARGINE_LISTINO6']].copy()
ricarichi_list5 = listini_sum[['RIC_20_LIST_5', 'RIC_30_LIST_5', 'RIC_35_LIST_5']].copy()
ricarichi_list6 = listini_sum[['RIC_20_LIST_6', 'RIC_30_LIST_6', 'RIC_35_LIST_6']].copy()

In [13]:
# Esportiamo i dati in un file Excel salvato localmente
writer = pd.ExcelWriter(report_file, engine='xlsxwriter')
margini.to_excel(writer, sheet_name='Margini')
ricarichi_list5.to_excel(writer, sheet_name='Ricarichi_LIST_5')
ricarichi_list6.to_excel(writer, sheet_name='Ricarichi_LIST_6')
writer.save()